# OpenHebrewBible (OHB) CSV Data to SQL Conversion

Eliran Wong used date from [ETCBC](https://github.com/ETCBC/bhsa) (Hebrew text BHSA, linguitic annotations, morphology, etc.), [OpenScriptures](https://github.com/openscriptures/morphhb) (Hebrew text WLC, Strong's numbers, morphology, etc.), and [Berean.bible](https://berean.bible) (interlinear translation, Berean Study Bible, etc.) to create a robust data repository called [OpenHebrewBible](https://github.com/eliranwong/OpenHebrewBible), consisting of CSV files that bridge the other three open-source projects.

I will take his compiled data file, [BHSA-with-extended-features.csv](https://github.com/eliranwong/OpenHebrewBible/blob/master/BHSA-with-extended-features.csv.zip), clean it, and convert it into a SQL database that I can use in my Flutter app. 

Later I will compare the converted SQL database to a BHSA SQL file, which can be downloaded [here](https://www.adambaker.org/bhsa.sqlite).

### Why uses OHB data when BHSA already exists?
Some features that could be useful from OHB data that aren't present in BHSA are:
- Strong's number mapped to each node in the BHS.
- Data to align the BHS text with KJV and BSB translations.
- Poetice devisions (not tested).
- BSB gloss for a more accurate rendering of each word. 

## Imports

In [ ]:
# Requirements: run in terminal. Change to 'pip' if on Windows OS. 
"""
pip3 install pandas
pip3 install numpy
pip3 install text-fabric
pip3 install jupyter
"""

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import copy
from tf.app import use
from IPython.display import display, HTML

pd.set_option('display.max_columns', None)

## The Data

The extended BHSA file consists of all the individual Hebrew words in the BHS text. It has 22 feature columns and uses tab-separeted delineation. All of the data consists of strings or positive integers. 

You can view the first two rows (1: column names, 2: data) of the file below.

In [20]:
# Convert the extended.csv file to a pandas dataframe.
csv_file = '../data_files/BHSA-with-extended-features.csv'
# Set low_memory to False to deal with unexpected data types. 
# Converts those data to NaN.
df = pd.read_csv(csv_file, sep='\t', low_memory=False)

# View the first two rows of our dataframe. 
display(HTML(df.head(n=1).to_html(index=False)))

BHSwordSort,paragraphMarker,poetryMarker,〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕,〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕,clauseID,clauseKind,clauseType,language,BHSwordPointed,BHSwordConsonantal,SBLstyleTransliteration,poneticTranscription,HebrewLexeme,lexemeID,StrongNumber,extendedStrongNumber,morphologyCode,morphologyDetail,ETCBCgloss,extendedGloss,〔BSBsort＠BSB〕
1,¶,NaN,〔1｜1｜1｜1〕,〔1｜1｜1｜1〕,c1,Verbal clauses,x-qatal-X clause,Hebrew,<heb>בְּ</heb><heb></heb>,<heb>ב</heb><heb></heb>,bĕ,bᵊ,<heb>בְּ</heb>,E70001,NaN,H9003,prep,preposition,in,in,〔1＠In〕


## Cleaning the Data

### Data to clean:
```
- 〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕: remove chars and place ints in new columns
- 〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕: remove chars and place ints in new columns
-  clauseID : remove 'c' prefix and convert to int
-  BHSwordPointed : remove html tags, place word and suffix in list
-  BHSwordConsonantal : remove html tags, place word and suffix in list
-  HebrewLexeme : remove html tags
- 〔BSBsort＠BSB〕: remove chars and place int and string in new columns
```

### 1. Clean Text and Clause Data

** **Important note:** certain nodes do not have a text value for *BHSwordPointed* or *BHSWordConsonantal* because of the nature of the Hebrew language. For example at BHS word node values 61-62 we have:
```
61  <heb>לָ</heb><heb></heb>     <heb>ל</heb><heb></heb>     <heb>לְ</heb>    H9005   prep    to	
62  <heb></heb><heb></heb>      <heb></heb><heb></heb>      <heb>הַ</heb>    H9009   art     the	〔51＠the〕
```
This if from a clause in Genesis 5:1, with the Hebrew: וַיִּקְרָא אֱלֹהִים לָאוֹר יוֹם

Node 62 is embedded into the word לָאוֹר, attached to the preoposition via a patach, but the *he* (the) doesn't appear consonantaly in the text.


##### Visualize pre-cleaned data

In [21]:
# Data before being cleaned.
display(HTML(
    df[
        ["BHSwordPointed", 
        "BHSwordConsonantal", 
        "HebrewLexeme", 
        "clauseID"]
    ].head().to_html(index=False))
)

BHSwordPointed,BHSwordConsonantal,HebrewLexeme,clauseID
<heb>בְּ</heb><heb></heb>,<heb>ב</heb><heb></heb>,<heb>בְּ</heb>,c1
<heb>רֵאשִׁ֖ית</heb><heb> </heb>,<heb>ראשית</heb><heb> </heb>,<heb>רֵאשִׁית</heb>,c1
<heb>בָּרָ֣א</heb><heb> </heb>,<heb>ברא</heb><heb> </heb>,<heb>ברא</heb>,c1
<heb>אֱלֹהִ֑ים</heb><heb> </heb>,<heb>אלהים</heb><heb> </heb>,<heb>אֱלֹהִים</heb>,c1
<heb>אֵ֥ת</heb><heb> </heb>,<heb>את</heb><heb> </heb>,<heb>אֵת</heb>,c1


##### Define functions

In [22]:
# Textual items in between word nodes, including paragraph markers, etc. 
text_extensions = {
    '', '׃', '׃ ׆ ס ', ' ס ', '׃ ׆ ', '׃ ', ' ׀ ',
    ' ', '׃ פ ', ' פ ', '׀ ', '׃ ׆ פ ', '־', '׃ ס '
}

# ---
# Function that takes a column name from the original df and
# returns cleaned text (word and extension) separated by |.
# Use: BHS pointed and consonantal text, all of which is of a format similar
# to : <heb>הָ</heb><heb></heb>. Be sure to update the df with the return value. 
def clean_text(col_name):
    cleaned_text = []
    # All of the junk html text present.
    remove_items = "/<arc>hebqrQR"
    # Either of these will appear between the word and extension.
    seperator = ["</heb><heb>", "</arc><arc>"]
    # Iterate over the original dataframe and clean the data. 
    for text_data in df[col_name]:
        # Place | at center so we can later split the text data. 
        for sep in seperator:
            if sep in text_data:
                text_data = text_data.replace(sep, '|')
        # Remove all extra items.
        for char in remove_items:
            if char in text_data:
                text_data = text_data.replace(char, "")

        # Note: I originally split each text and stored it in a list before 
        # appending to cleaned_text, but that caused an error when uploading 
        # to SQL because it needed an actual data type (e.g., string).
        
        # Add a text separated by | to cleaned text where pre '|' is 
        # a Heb word and post '|' is the extension.
        cleaned_text.append(text_data)
    
    return cleaned_text


# ---
# Clean the text in the HebrewLexem column, all of which is
# in a format similar to: <heb>הָ</heb>. 
def clean_lexemes(col_name):
    cleaned_text = []
    # Read comments from clean_text()
    remove_items = "/<arc>hebqrQR"
    # Iterate over the original dataframe and clean the data. 
    for text_data in df[col_name]:
        # Remove all extra items.
        for char in remove_items:
            if char in text_data:
                text_data = text_data.replace(char, "")
        # Add the lexeme to the cleaned data.
        cleaned_text.append(text_data)

    return cleaned_text


# ---
# All clause data is of the format: c12. Remove the 'c's
# in the clause data and convert to int type. 
def clean_clauses(col_name):
    cleaned_ids = []
    # Iterate over the original dataframe and clean the data. 
    for clause in df[col_name]:
        cleaned_ids.append(int(clause.strip("c")))
        
    return cleaned_ids

##### Call the functions -> update dataframe

In [23]:
# Update the data frame with the cleaned text and clauses. 
df["BHSwordPointed"] = clean_text("BHSwordPointed")
df["BHSwordConsonantal"] = clean_text("BHSwordConsonantal")
df["HebrewLexeme"] = clean_lexemes("HebrewLexeme")
df["clauseID"] = clean_clauses("clauseID")

##### Visualize cleaned data

In [24]:
# Print the head with the cleaned data.
display(HTML(
    df[
        ["BHSwordPointed", 
        "BHSwordConsonantal", 
        "HebrewLexeme", 
        "clauseID"]
    ].head().to_html(index=False))
)

BHSwordPointed,BHSwordConsonantal,HebrewLexeme,clauseID
בְּ|,ב|,בְּ,1
רֵאשִׁ֖ית|,ראשית|,רֵאשִׁית,1
בָּרָ֣א|,ברא|,ברא,1
אֱלֹהִ֑ים|,אלהים|,אֱלֹהִים,1
אֵ֥ת|,את|,אֵת,1


### 2. Expand KJV, BHS, and BSB columns

In the original 22 columns there are three features that consist of concatenated values:

- 〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕:〔1｜1｜1｜1〕  
- 〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕:〔1｜1｜1｜1〕
- 〔BSBsort＠BSB〕:〔1＠In〕

I will convert each of them to new dataframes with separate columns for each value, and then merge them back into the original dataframe.

##### Visualize pre-cleaned data

In [25]:
# Data before being cleaned.
display(HTML(
    df[
        ['〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕', 
        '〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕',
        '〔BSBsort＠BSB〕']
    ].head().to_html(index=False))
)

〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕,〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕,〔BSBsort＠BSB〕
〔1｜1｜1｜1〕,〔1｜1｜1｜1〕,〔1＠In〕
〔1｜1｜1｜1〕,〔1｜1｜1｜1〕,〔2＠the beginning〕
〔1｜1｜1｜1〕,〔1｜1｜1｜1〕,〔4＠created〕
〔1｜1｜1｜1〕,〔1｜1｜1｜1〕,〔3＠God〕
〔1｜1｜1｜1〕,〔1｜1｜1｜1〕,NaN


##### Define functions

In [26]:
# Where the value is a list, convert the original 
# column to len(list) new columns. 
updated_col_names = {
    'BHSwordSort': # no cleaning
        'BHS_wordNode', 
    '〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕': # cleaned
        ['KVJvsNode', 'KJVbook', 'KJVchapter', 'KJVverse'], 
    '〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕': # cleaned
        ['BHSvsNode', 'BHSbook', 'BHSchapter', 'BHSverse'], 
    'BHSwordPointed': # cleaned
        'BHS_wordPointed', 
    'BHSwordConsonantal': # no cleaning
        'BHS_wordConsonantal',
    'poneticTranscription': # no cleaning
        'phoneticTranscription',
    '〔BSBsort＠BSB〕': # cleaned
        ['BSBglossNode', 'BSBgloss']
}

# ---
# Function that takes a column name from 
# the original df and returns cleaned data.
# Use: convert the KJF ref or BHS ref column to new dataframe. 
def clean_references(col_name):
    # Create a dict for each new name to the new values.
    new_names = updated_col_names[col_name]
    cleaned_data = {name:[] for name in new_names}
    # Iterate over the original dataframe and clean the data. 
    for ref_data in df[col_name]:
        # Remove the outsides of 〔1｜1｜1｜1〕.
        ref_data = ref_data.strip('〕〔')
        # Split 1｜1｜1｜1 and convert each item to an int.
        ref_data = [int(data) for data in ref_data.split('｜')]
        # Add data to the dictionary. 
        cleaned_data[new_names[0]].append(ref_data[0]) # vs node
        cleaned_data[new_names[1]].append(ref_data[1]) # book
        cleaned_data[new_names[2]].append(ref_data[2]) # chapter
        cleaned_data[new_names[3]].append(ref_data[3]) # verse
   
    # Convert the dictionary to a dataframe and return.
    new_df = pd.DataFrame(cleaned_data)
    return new_df

# ---
# Clean the BSB gloss data and store in a new dataframe. 
def clean_gloss(col_name):
    # Create a dict for each new name to the new values.
    new_names = updated_col_names[col_name]
    cleaned_data = {name:[] for name in new_names}
    # Iterate over the original dataframe and clean the data. 
    for gloss_data in df[col_name]:
        # Catch edge cases where gloss_data is NaN.
        if isinstance(gloss_data, str):
            # Remove the outsides of 〔1＠In〕.
            gloss_data = gloss_data.strip('〕〔')
            # Split 1＠In and convert first item to an int.
            gloss_data = gloss_data.split('＠')
            
            # For some reason, gloss node 237839 is split into 
            # decimals .1 and .2, which is why I am using a float. 

            # Add data to the dictionary. 
            gloss_data[0] = float(gloss_data[0])
            cleaned_data[new_names[0]].append(gloss_data[0]) # gloss node
            cleaned_data[new_names[1]].append(gloss_data[1]) # gloss
        else:
            cleaned_data[new_names[0]].append(gloss_data) # gloss node
            cleaned_data[new_names[1]].append(gloss_data) # gloss

    # Convert the dictionary to a dataframe and return.
    new_df = pd.DataFrame(cleaned_data)
    return new_df

##### Call the functions -> new dataframes

In [27]:
# Clean the reference data and store in two new dataframes. 
KJV_ref_df = clean_references(
    '〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕')
BHS_ref_df = clean_references(
    '〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕')
BSB_gloss_df = clean_gloss(
    '〔BSBsort＠BSB〕')

##### Visualize cleaned data

In [28]:
# Print the head of the cleaned data.
display(HTML(
    pd.concat(
        [KJV_ref_df, 
        BHS_ref_df, 
        BSB_gloss_df],
        axis=1
    ).head().to_html(index=False))
)

KVJvsNode,KJVbook,KJVchapter,KJVverse,BHSvsNode,BHSbook,BHSchapter,BHSverse,BSBglossNode,BSBgloss
1,1,1,1,1,1,1,1,1.0,In
1,1,1,1,1,1,1,1,2.0,the beginning
1,1,1,1,1,1,1,1,4.0,created
1,1,1,1,1,1,1,1,3.0,God
1,1,1,1,1,1,1,1,NaN,NaN


### 3. Rename columns and combine dataframes

##### Define functions

In [29]:
# ---
# Drop the three columns that I expanded into new dataframes.
def drop_old_data(dateframe):
    dateframe = dateframe.drop(
        columns=[
        '〔KJVverseSort｜KJVbook｜KJVchapter｜KJVverse〕', 
        '〔BHSverseSort｜BHSbook｜BHSchapter｜BHSverse〕',
        '〔BSBsort＠BSB〕']
    )
    return dateframe

# ---
# Rename certaine columns and add the three 
# new dataframes for a final df output. 
def combine_data():
    df_copy = copy.deepcopy(df)
    updated_df = pd.DataFrame()
    # Make sure the replaced data gets dropped. 
    if "〔BSBsort＠BSB〕" in df_copy.columns:
        df_copy = drop_old_data(df_copy)
    # Rename the other columns and build updated_df.
    for column in df_copy:
        # Simply add the renamed column if it's in our dictionary. 
        if column in updated_col_names:
            updated_df[updated_col_names[column]] = df[column]
        # If column isn't in our dictionary, add new dfs or
        # simply add the current column. 
        else:
            # If at the column before 〔KJVverseSort..., 
            # add the new reference dataframes.
            if column == "poetryMarker":
                updated_df = pd.concat(
                    [updated_df, 
                    df_copy[column], 
                    KJV_ref_df, 
                    BHS_ref_df], 
                    axis=1)
            # If at the column before 〔BSBsort..., 
            # add the new BSB dataframe.
            elif column == "extendedGloss":
                updated_df = pd.concat(
                    [updated_df, 
                    df_copy[column], 
                    BSB_gloss_df], 
                    axis=1)
            # Otherwise add the current column 
            # from the original dataframe.
            else:
                updated_df[column] = df_copy[column]

    return updated_df

##### Call function -> combined dataframe

In [30]:
# Store the combined data in a new dataframe. 
updated_data = combine_data()

### 4. Finally - Replace NaN values with "" to match string data.

##### Visualize pre-cleaned data

In [31]:
# Display the labeled data.
display(HTML(updated_data.head().to_html(index=False)))

BHS_wordNode,paragraphMarker,poetryMarker,KVJvsNode,KJVbook,KJVchapter,KJVverse,BHSvsNode,BHSbook,BHSchapter,BHSverse,clauseID,clauseKind,clauseType,language,BHS_wordPointed,BHS_wordConsonantal,SBLstyleTransliteration,phoneticTranscription,HebrewLexeme,lexemeID,StrongNumber,extendedStrongNumber,morphologyCode,morphologyDetail,ETCBCgloss,extendedGloss,BSBglossNode,BSBgloss
1,¶,NaN,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,בְּ|,ב|,bĕ,bᵊ,בְּ,E70001,NaN,H9003,prep,preposition,in,in,1.0,In
2,NaN,NaN,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,רֵאשִׁ֖ית|,ראשית|,rēšît,rēšˌîṯ,רֵאשִׁית,E70002,H7225,H7225,subs.f.sg.a,"noun, feminine, singular, absolute",beginning,beginning,2.0,the beginning
3,NaN,NaN,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,בָּרָ֣א|,ברא|,bārā,bārˈā,ברא,E70003,H1254,H1254,verb.qal.perf.p3.m.sg,"verb, qal, perfect, third person, masculine, singular",create,[he]+ create,4.0,created
4,NaN,NaN,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,אֱלֹהִ֑ים|,אלהים|,ʾĕlōhîm,ʔᵉlōhˈîm,אֱלֹהִים,E70004,H430,H430,subs.m.pl.a,"noun, masculine, plural, absolute",god(s),god [pl.],3.0,God
5,NaN,NaN,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,אֵ֥ת|,את|,ʾēt,ʔˌēṯ,אֵת,E70005,H853,H853,prep,preposition,[object marker],[object marker],NaN,NaN


##### Define function

In [32]:
# All string type columns in updated_data that have NaNs present.
cols_with_NaN = [
    "paragraphMarker",
    "poetryMarker",
    "SBLstyleTransliteration",
    "phoneticTranscription",
    "StrongNumber",
    "BSBgloss"
]

# ---
# Function to replace NaNs with values. 
def replace_NaNs():
    for col in cols_with_NaN:
        updated_data[col] = updated_data[col].replace({np.nan: ""})

##### Call function -> clean dataframe

Note: I retained NaNs in the *BSBglossNode* column since it stores floats.

In [33]:
# Call the function.
replace_NaNs()

##### Visualize the final cleaned dataframe

In [34]:
# Display our newly cleaned and labeled data.
display(HTML(updated_data.head().to_html(index=False)))

BHS_wordNode,paragraphMarker,poetryMarker,KVJvsNode,KJVbook,KJVchapter,KJVverse,BHSvsNode,BHSbook,BHSchapter,BHSverse,clauseID,clauseKind,clauseType,language,BHS_wordPointed,BHS_wordConsonantal,SBLstyleTransliteration,phoneticTranscription,HebrewLexeme,lexemeID,StrongNumber,extendedStrongNumber,morphologyCode,morphologyDetail,ETCBCgloss,extendedGloss,BSBglossNode,BSBgloss
1,¶,,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,בְּ|,ב|,bĕ,bᵊ,בְּ,E70001,,H9003,prep,preposition,in,in,1.0,In
2,,,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,רֵאשִׁ֖ית|,ראשית|,rēšît,rēšˌîṯ,רֵאשִׁית,E70002,H7225,H7225,subs.f.sg.a,"noun, feminine, singular, absolute",beginning,beginning,2.0,the beginning
3,,,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,בָּרָ֣א|,ברא|,bārā,bārˈā,ברא,E70003,H1254,H1254,verb.qal.perf.p3.m.sg,"verb, qal, perfect, third person, masculine, singular",create,[he]+ create,4.0,created
4,,,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,אֱלֹהִ֑ים|,אלהים|,ʾĕlōhîm,ʔᵉlōhˈîm,אֱלֹהִים,E70004,H430,H430,subs.m.pl.a,"noun, masculine, plural, absolute",god(s),god [pl.],3.0,God
5,,,1,1,1,1,1,1,1,1,1,Verbal clauses,x-qatal-X clause,Hebrew,אֵ֥ת|,את|,ʾēt,ʔˌēṯ,אֵת,E70005,H853,H853,prep,preposition,[object marker],[object marker],NaN,


## Importing the Dataframe Into a SQL Database

Adam Baker used C++ to convert the BHSA corpus into a sql database. He outlines his work on his [site](https://www.adambaker.org/text-fabric-format.php) where he links the sql file as well as his GitHub repository. Rather than create a new database, I will add Eliran's data as new table in Adam's database.

In [35]:
# Adam's sql file.
sql_file = '../data_files/bhsa.sqlite'
con = sqlite3.connect(sql_file) 
# Convert the dataframe to a table called extended_data in the bhsa database. 
updated_data.to_sql(
    "extended_data", 
    con=con, 
    if_exists='replace', 
    index=False)

## Test OHB Data Against BHSA

I want to compare values in the OHB data to the BHSA data, especially text values, to make sure that the data is usable and accurate. 

In [36]:
# Import the BHSA API
A = use('bhsa', hoist=globals())

This is Text-Fabric 9.1.1
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


### Compare consonantal, pointed, and lexeme word forms

##### Load SQL data

In [39]:
# To execute queries. 
sql_file = '../data_files/bhsa.sqlite'
con = sqlite3.connect(sql_file) 

# Words from new SQL database.
sql_pointed = con.execute(
    "SELECT BHS_wordPointed FROM extended_data"
    ).fetchall()
sql_consonant = con.execute(
    "SELECT BHS_wordConsonantal FROM extended_data"
    ).fetchall()
sql_lexeme = con.execute(
    "SELECT HebrewLexeme FROM extended_data"
    ).fetchall()

# Convert tuples to lists.
sql_pointed = [i[0] for i in sql_pointed]
sql_consonant = [i[0] for i in sql_consonant]
sql_lexeme = [i[0] for i in sql_lexeme]

##### Define Functions

In [40]:
# ---
# Function that goes through all word-type nodes in the 
# BHSA and compares them to words in Eliran's data. 
def check_text():
    # Values used to realign OHB data to 
    # BHSA where rows are missing.
    j = -1
    k = 0
    # Iterate over all words in BHSA. Stop at 426581,
    # the max node in SQL, there are a few more in BHS.
    for word in F.otype.s('word')[:426581]:

        # BHSA node count starts at 1. 
        i = word + j
        # Prevent closed loop.
        i = i+1 if i == k else i

        # -- Make sure the data stays aligned --
        # See "Textual mismatch notes" to understand. 
        # Catch missing node in OHB data.
        if word == 16564:
            j -= 1
        # Catch instances of doubles.
        dup_check = sql_consonant[i].split('|')[0]
        if dup_check in doubles:
            j -= 1
            k = i
            doubles[dup_check].append(word)
        # OHB node 392490 takes the place of three 
        # nodes in BHSA: 392490 - 392492, so we 
        # increment j by 2 to preserve alignment.
        if dup_check == "חצי המנחות":
            j -= 2
            k = i
        
        # -- Perform comparisons and update lists --
        # Compare pointed.
        sql_pt =  sql_pointed[i].split('|')[0]
        bhs_pt = F.g_word_utf8.v(word)
        if sql_pt != bhs_pt:
            pointed_mismatches.append(
                f"{word} - BHSA:{bhs_pt} | SQL:{sql_pt}")
        # Compare pointed extension. 
        sql_pt_xt = sql_pointed[i].replace('|', '')
        bhs_pt_xt = T.text(word, fmt='text-orig-full')
        if sql_pt_xt != bhs_pt_xt :
            pointed_ext_mismatches.append(
                f"{word} - BHSA:{bhs_pt_xt} | SQL:{sql_pt_xt}")
        # Compare consonsantal.
        sql_cs =  sql_consonant[i].split('|')[0]
        bhs_cs = F.g_cons_utf8.v(word)
        if bhs_cs != sql_cs:
            consonant_mismatches.append(
                f"{word} - BHSA:{bhs_cs} | SQL:{sql_cs}")
        # Compare consonantal extension. 
        sql_cs_xt = sql_consonant[i].replace('|', '')
        bhs_cs_xt = T.text(word, fmt='text-orig-plain')
        if bhs_cs_xt != sql_cs_xt:
            consonant_ext_mismatches.append(
                f"{word} - BHSA:{bhs_cs_xt} | SQL:{sql_cs_xt}")
        # Compare lexeme.
        sql_lex = sql_lexeme[i]
        bhs_lex = F.g_lex_utf8.v(word)
        if bhs_lex != sql_lex:
            lexeme_mismatches.append(
                f"{word} - BHSA:{bhs_lex} | SQL:{sql_lex}")


# ---
# Convert all of the mismatch data into a dataframe. 
def display_mismatches():
    # Create individual dataframes.
    pointed = pd.DataFrame(
        pointed_mismatches, 
        columns =['Pointed'])
    pointed_xt = pd.DataFrame(
        pointed_ext_mismatches, 
        columns =['Pointed-ext'])
    consonantal = pd.DataFrame(
        consonant_mismatches, 
        columns =['Consonantal'])
    consonantal_xt = pd.DataFrame(
        consonant_ext_mismatches, 
        columns =['Consonantal-ext'])
    lexeme = pd.DataFrame(
        lexeme_mismatches, 
        columns =['Lexeme'])
    # Put them together.
    text_mismatches = pd.concat(
                    [pointed, 
                    pointed_xt, 
                    consonantal, 
                    consonantal_xt,
                    lexeme], 
                    axis=1)
    
    return text_mismatches

##### Call functions -> populate mismatch data

In [41]:
# Catch all instances of ארם נהרים.
doubles = {
    "ארם נהרים": [],
    "ארם מעכה": [] 
}

# Lists to hold mismatches.
pointed_mismatches = []
pointed_ext_mismatches = []
consonant_mismatches = []
consonant_ext_mismatches = []
lexeme_mismatches = []

# Compare data.
check_text()

##### Create dataframe to visualize mismatches

In [42]:
# Create mismatch dataframe.
text_mismatches = display_mismatches()

##### Visualize the mismatch data

In [43]:
# Display head.
display(HTML(text_mismatches.head(10).to_html(index=False)))

Pointed,Pointed-ext,Consonantal,Consonantal-ext,Lexeme
1 - BHSA:בְּ | SQL:רֵאשִׁ֖ית,1 - BHSA:בְּ | SQL:רֵאשִׁ֖ית,1 - BHSA:ב | SQL:ראשית,1 - BHSA:ב | SQL:ראשית,1 - BHSA:בְּ | SQL:רֵאשִׁית
3897 - BHSA:הוצא | SQL:הַיְצֵ֣א,7138 - BHSA:בֵינֶֽיׄךָ׃ | SQL:בֵינֶֽיׄכָ׃,2 - BHSA:ראשׁית | SQL:ראשית,2 - BHSA:ראשׁית | SQL:ראשית,3 - BHSA:בָּרָא | SQL:ברא
4420 - BHSA:אהלה | SQL:אָהֳלֹֽו,11325 - BHSA:אֲרַ֥ם | SQL:אֲרַ֥ם נַֽהֲרַ֖יִם,7 - BHSA:שׁמים | SQL:שמים,7 - BHSA:שׁמים | SQL:שמים,4 - BHSA:אֱלֹה | SQL:אֱלֹהִים
5645 - BHSA:אהלה | SQL:אָהֳלֹ֑ו,11326 - BHSA:נַֽהֲרַ֖יִם | SQL:אֶל־,20 - BHSA:חשׁך | SQL:חשך,11 - BHSA:ארץ׃ | SQL:ארץ,7 - BHSA:שָּׁמַי | SQL:שָׁמַיִם
5912 - BHSA:אהלה | SQL:אָֽהֳלֹו֙,16564 - BHSA: | SQL:חֲלֹ֖ום,57 - BHSA:חשׁך | SQL:חשך,20 - BHSA:חשׁך | SQL:חשך,10 - BHSA:הָ | SQL:הַ
6246 - BHSA:צביים | SQL:צְבֹויִ֔ם,71532 - BHSA:אַׄהֲׄרֹ֛ׄןׄ | SQL:אַׄהֲׄרֹ֛ׄנׄ,68 - BHSA:חשׁך | SQL:חשך,21 - BHSA:על־ | SQL:על,11 - BHSA:אָרֶץ | SQL:אֶרֶץ
6354 - BHSA:צביים | SQL:צְבֹויִ֔ם,76767 - BHSA:מַּחֲנֶֽה׃ נ ס | SQL:מַּחֲנֶֽה׃ ׆ ס,96 - BHSA:יעשׂ | SQL:יעש,28 - BHSA:על־ | SQL:על,13 - BHSA:הָ | SQL:הַ
7138 - BHSA:בֵינֶֽיׄךָ | SQL:בֵינֶֽיׄכָ,76795 - BHSA:יִשְׂרָאֵֽל׃ נ פ | SQL:יִשְׂרָאֵֽל׃ ׆ פ,106 - BHSA:אשׁר | SQL:אשר,31 - BHSA:מים׃ | SQL:מים,14 - BHSA:אָרֶץ | SQL:אֶרֶץ
11325 - BHSA:אֲרַ֥ם | SQL:אֲרַ֥ם נַֽהֲרַ֖יִם,88141 - BHSA:עִשָּׂרֹוׄן֙ | SQL:עִשָּׂרֹוׄנ֙,116 - BHSA:אשׁר | SQL:אשר,38 - BHSA:יהי־ | SQL:יהי,15 - BHSA:הָי | SQL:היה
11326 - BHSA:נַֽהֲרַ֖יִם | SQL:אֶל,105982 - BHSA:אֲרַ֥ם | SQL:אֲרַ֥ם נַהֲרַ֖יִם,131 - BHSA:שׁמים | SQL:שמים,39 - BHSA:אור׃ | SQL:אור,16 - BHSA:תֹהוּ | SQL:תֹּהוּ


##### Save dataframe locally

In [44]:
# Drop non-meaningful data.
# (See notes at the end to see why they are meaningless)
text_mismatches = text_mismatches.drop(
    columns=[
        "Pointed",
        "Consonantal",
        "Consonantal-ext",
        "Lexeme"])

# Drop the extra rows. 
text_mismatches.dropna(
    # subset = ["Pointed"],
    inplace=True)

In [45]:
# Save as a txt file.
save_to = "../data_files/mismatches.txt"
text_mismatches.to_csv(
    save_to,
    sep='\t',
    index=False)

##### Function to visualize BHSA nodes after comparing data

In [46]:
# ---
# Function to get BHSA words in a range of the
# mismatched OHB node. 
def bhsa_in_range(i):
    nodes = []
    # start preceeding nodes and end following nodes.
    start= -6
    end= 2
    for j in range(start, end):
        nodes.append(i+j)
    # Display the node in it's Scripture context.
    for n in nodes:
        print("Node:", n)
        A.plain(n)

# Call function
# bhsa_in_range(16562)

### Textual mismatch notes
```Note: all references to 'node' follow the *BHA_wordNode* value. If there is a mismatch I will include BHSA node values in parentheses.```

**Pointed text differences**
- Places where the BHSA lacks text markings that are included in Leningrad Codex (WLC), following the BHS, OHB chose the WLC form. For example, at node 4420, in Gen 9:21, [BHS renders](https://www.academic-bible.com/en/online-bibles/biblia-hebraica-stuttgartensia-bhs/read-the-bible-text/bibel/text/lesen/stelle/1/90001/99999/ch/df5b28ecb57b22ae25d0ce157bea69ca/) tent as אהלה while [WLC renders](https://www.bible.com/bible/904/GEN.9.WLC) it as אָהֳלֹֽה. *Note that the text displayed using the BHSA API function T.text(4420) actually does display אָהֳלֹֽו.* However, OHB's rendering of node 4420 matches neither of these because his ends in a *vav*: אָהֳלֹֽו.
- There are places where OHB fails to place the ending consonant in its sofit form. E.g., at node 7138 it has בֵינֶֽיׄכָ instead of the BHSA's בֵינֶֽיׄךָ. 
- **Major Error:** Non-aligned & missing data -> offset.
    - At node 11325 OHB combines two words (BHSA 11325, 11326), causing all the remaining nodes to be offset by 1. OHB has אֲרַ֥ם נַֽהֲרַ֖יִם where the BHSA has אֲרַ֥ם. BHSA node 11326 is נַֽהֲרַ֖יִם. This same thing also occurs at nodes 105980, 128870, 320251, and 401286 (BHSA 105982, 128873, 320255, 401293; notice the offset increases each time, by 2, 3, ...). This offset occurs similarly at 401289 with the words אֲרַ֤ם מַעֲכָה֙ (BHSA 401297 and 401298).
    - At nodes 16562 to 16563 (BHSA 16563, 16564, 16565) the OHB data jumps from בַּ to חֲלֹ֖ום, missing a row in the middle where BHSA has a node (BHSA 16564) for the embedded *he* (הָ) in בַּ. This again causes an offset. 
    - At node 392485 (BHSA 392490), the OHB has חֲצִי הַמְּנֻחֹות which takes the place of three nodes in BHSA (392490, 392491, 392492).

**Consonantal text differences:** 
- The BHSA includes the difference between *shin* and *sin*, whereas OHB data doesn't. For example, at node 72 the BHSA has יעשׂ while the SQL data has יעש. In this, OHB reflects common practice with consonantal Hebrew, e.g., [Sefaria](https://www.sefaria.org/Genesis.1.12?lang=bi&aliyot=0).
- The BHSA includes extensions (see *text_extensions* list in the "Clean Text and Clause Data" section), whereas OHB only preserves spaces. For example, at node 11 (the end of Gen 1:1), the BHSA has ארץ׃ while the OHB has ארץ without the sof pasuk (׃).

**Lexeme text differences:** The BHSA differs with both letters and vowels. Some example nodes are:
```
- 95  BHSA:וַ   | SQL:וְ 
- 96  BHSA:עַשׂ  | SQL:עשׂה 
- 97  BHSA:אֱלֹה | SQL:אֱלֹהִים 
- 98  BHSA:אֶת  | SQL:אֵת 
- 99  BHSA:הָ   | SQL:הַ 
```

### Summary
We end up with about 221 differences for the pointed text. Some are significant, but many have to do with accent marks. (**Note** that "significant" refers to the ability to query items associated with a specific word node. The entire BHS visible text is still present to render in an app, but the lack of node-id alignment with BHSA could make certain queries difficult). There are many more differences with the consonants, which are not of concern because OHB follows the common form. There are many lexeme differences which may be of concern. 